In [3]:
from itertools import count
from re import L
from turtle import left, right
import numpy as np
import pandas as pd
import openpyxl as op

In [4]:
# import data
bulk_list_df = pd.read_excel('Bulk Location Dimensions 5-5-22.xlsx', sheet_name='Bulk Locs inches')
full_df = pd.read_excel('Product_By_Location_Report 4-22-22.xlsx', sheet_name='Report 1', skiprows=7)
stack_df = pd.read_excel('Stacking Allowances 4-25-22.xlsx', sheet_name='Max Stack List', skiprows=1)
article_master_df = pd.read_excel('SPWR_ArticleMaster_(KF)_and_PackStructure_(PD)_Report.xlsx',
                                  sheet_name='SPWR Article Master')
# drop empty columns created by cells unmerged in Excel and other unused columns
full_df = full_df.drop(
    ['Unnamed: 0', 'Depot Code', 'Unnamed: 3', 'Unnamed: 14', 'Unnamed: 16', 'Article Ref 3', 
     'Article Date 1', 'Article Date 2', ], axis=1)
# numbers come formatted with commas which needs to be cleaned to allow for numerical operations
full_df = full_df.replace(',', '', regex=True)
article_master_df = article_master_df.replace(',', '', regex=True)
bulk_list_df = bulk_list_df.replace(',', '', regex=True)
stack_df = stack_df.replace(',', '', regex=True)
# remove trailing and leading spaces from applicable data so that identical values between tables 
# can be recognized
bulk_list_df['Whse Location'] = bulk_list_df['Whse Location'].str.strip()
full_df['Whse Location'] = full_df['Whse Location'].str.strip()
full_df['Article ID'] = full_df['Article ID'].str.strip()
article_master_df['Article ID'] = article_master_df['Article ID'].str.strip()
stack_df['Article ID'] = stack_df['Article ID'].str.strip()
# create dataframe bulk_df as subset of full_df containing only bulk locations
bulk_df = full_df
bulk_df = bulk_df.merge(bulk_list_df, how='right', on='Whse Location')
bulk_df = bulk_df.sort_values('Whse Location')

In [ ]:
# find volume utilization in bulk area
def find_bulk_utilization():
    bulk_volume_used = bulk_df['Case Volume Actual'].sum()
    # bulk_list has units in inches, convert to ft by dividing by 1728 in3/ft3 to match bulk_volume_used
    bulk_volume_available = bulk_list_df['Avail Volume (in3)'].sum() / 1728
    bulk_utilization = bulk_volume_used / bulk_volume_available
    print("Bulk Utiliztion", round((100 * bulk_utilization), 2), "%")
    return

In [ ]:
# create partially filled loc report
# create column with qty of each item in each loc
count_df = bulk_df
count_df['Count'] = 0
count_df = count_df.groupby(['Whse Location']).Count.count().reset_index()
count_df = count_df.drop_duplicates(subset=['Whse Location'])
bulk_df = bulk_df.drop_duplicates(subset=['Whse Location'])
bulk_df = bulk_df.reset_index()
count_df = count_df.reset_index()
bulk_df = bulk_df.merge(count_df, how='left', on='Whse Location')
bulk_df = bulk_df.drop(['index_x', 'Count_x', 'Whse Zone_y', 'index_y'], axis=1)
bulk_df = bulk_df.rename(columns={'Whse Zone_x': 'Whse Zone', 'Count_y': 'Qty in Loc'})
bulk_df = bulk_df.sort_values('Whse Location')

# add product dimensions
bulk_article_df = article_master_df[
    ['Article ID', 'Article Description', 'PL Length', 'PL Width', 'PL Height', 'PL Volume']]
bulk_df = bulk_df.merge(bulk_article_df, how='left', on='Article ID')
# create column for maximum number of items that can fit within each location
# still need to add rounding up all columns with number of columns as zero to 1
bulk_df['Max Columns of Item'] = (bulk_df['Avail Width (in)'] / (bulk_df['PL Width'] * 12)).apply(np.floor)
bulk_df['Max Columns of Item'].replace(0, 1)
bulk_df['Max Rows of Item'] = (bulk_df['Avail Depth  (in)'] / (bulk_df['PL Length'] * 12)).apply(np.floor)
bulk_df = bulk_df.merge(stack_df, how='left', on='Article ID')
# bulk_df['Max Stacks of Item'] = (bulk_df['Avail Height (in)']/(bulk_df['PL Height']*12)).apply(np.floor) bulk_df[
# 'Max Stacks of Item'] = bulk_df['Article ID'].apply(lambda x: (bulk_df['Avail Height (in)']/(bulk_df['PL
# Height']*12)).apply(np.floor) if bulk_df['Article ID'].str.contains('PV', case=False, regex=True) else 1) create
# column for number of additional items that could be added to each location
bulk_df['Max Item Qty'] = bulk_df['Max Columns of Item'] * bulk_df['Max Rows of Item'] * bulk_df['Max Stack']
bulk_df['Fillable Item Qty'] = bulk_df['Max Item Qty'] - bulk_df['Qty in Loc']
# restructure dataframe to be easier to read

In [ ]:
# export partially filled loc report
bulk_df.to_csv(r'C:\Python Apps\Space Utilization\Bulk App\Output Files\Test.csv', index=False)